In [34]:
import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.
import re # Регулярные выражения.
import html5lib

import numpy as np
import pandas as pd
from tqdm import tqdm

import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver import ActionChains

In [8]:
regularForHref = r'(href=\"([^"]*)\")'
regularExpForHref = re.compile(regularForHref)
hrefArray = []
titleArray = []

In [36]:
result = pd.DataFrame(columns=['href','title','text', 'theme'])

In [25]:
texts = []
hrefs = []
titles = []

In [15]:
themes = ['military', 'economy', 'columnist', 'dk']
indexes = [0, 1, 2, 3]
themeIndexes = dict(zip(indexes, themes))
themeIndexes

{0: 'military', 1: 'economy', 2: 'columnist', 3: 'dk'}

In [12]:
driver = webdriver.Chrome()

In [26]:
for theme in themes:
  driver.get(f"https://profile.ru/{theme}/")
  count = 0
  while count < 50:
    ActionChains(driver).scroll_by_amount(delta_x=0, delta_y=9999).perform()
    time.sleep(5)
    count+=1
    
  articles = driver.find_elements(By.CLASS_NAME, 'article__title')
  
  for article in tqdm(articles):
    titles.append(article.text)
    hrefs.append(article.get_attribute('href'))
    
  print(len(hrefs))
    

508
1016
1257
1775


508 m
508 e
241 c
518 dk

In [27]:
len(titles)

1775

In [30]:
hrefs[0]

'https://profile.ru/news/military/kalashnikov-vypustit-partiju-obnovlennyh-ak-12-vo-vtoroj-polovine-goda-1330609/'

In [31]:
for line in tqdm(hrefs):
  request = requests.get(line)
  bsArticle = BeautifulSoup(request.content, 'html5lib')
  article = bsArticle.find(name='div', attrs={'class':'micromarking'})
  texts.append(article.text)

In [33]:
len(texts)

1775

In [46]:
result['href'] = hrefs
result['title'] = titles
result['text'] = texts

result.to_csv('news.CSV', index=False)



In [48]:
result.index

RangeIndex(start=0, stop=1775, step=1)

In [58]:
zeros = np.zeros(508)
ones = np.ones(508)
twos = np.array([2 for i in range(241)])
threes = np.array([3 for i in range(518)])
    

In [61]:
general  = np.concatenate([zeros, ones, twos, threes])
general.shape

(1775,)

In [63]:
result['theme'] = general
result

,href,title,text,theme
0,https://profile.ru/news/military/kalashnikov-v...,"""Калашников"" выпустит партию обновленных АК-12...",\n Во втором полуго...,0.0
1,https://profile.ru/news/military/kalashnikov-n...,"""Калашников"" начал поставлять снайперскую винт...",\n Поставки новейше...,0.0
2,https://profile.ru/military/chernyj-maj-kak-ne...,"""Черный май"": как немецкие подводники потерпел...",\n 24 мая исполняет...,0.0
3,https://profile.ru/news/military/v-rf-prodolzh...,В РФ продолжают испытывать новый ракетный комп...,\n В России продолж...,0.0
4,https://profile.ru/military/za-chto-komandir-s...,"За что командир советской подлодки ""Щ-408"" пол...",\n 22 мая исполняет...,0.0
...,...,...,...,...
1770,https://profile.ru/news/dk/ugmk/shaaz-vypustil...,ШААЗ выпустил первый отечественный 14-тонный п...,\n «Шадринский авто...,3.0
1771,https://profile.ru/news/dk/rosterminalugol/dey...,Деятельность «Ростерминалугля» признана эколог...,\n «Ростерминалугол...,3.0
1772,https://profile.ru/dk/ugmk/perspektiva-dlya-za...,«Перспектива» для заслуженного отдыха,\n Почему у нашего ...,3.0
1773,https://profile.ru/news/dk/kru/kuzbassrazrezug...,«Кузбассразрезуголь» создал первый профстандар...,\n «Кузбассразрезуг...,3.0


In [64]:
result.to_csv('news.CSV', index=False)